In [1]:
library("data.table")

Warning message:
"package 'data.table' was built under R version 4.1.2"


In [2]:
# prepare dummy input

var_a <- c(
	1,
	2,
	3,
	4,
	2,
	3,
	1,
	5,
	3,
	4,
	6,
	4,
	3,
	7,
	5,
	6,
	4,
	5
)

var_b <- c(
	3,
	2,
	1,
	3,
	5,
	6,
	5,
	6,
	4,
	7,
	9,
	4,
	3,
	2,
	3,
	1,
	3,
	2
)

slk_from <- c(
	0.000,
	0.010,
	0.020,
	0.030,
	0.040,
	0.050,
	0.060,
	0.070,
	0.080,
	0.090,
	0.100,
	0.110,
	0.120,
	0.130,
	0.140,
	0.150,
	0.160,
	0.170
)

slk_to <- c(
	0.010,
	0.020,
	0.030,
	0.040,
	0.050,
	0.060,
	0.070,
	0.080,
	0.090,
	0.100,
	0.110,
	0.120,
	0.130,
	0.140,
	0.150,
	0.160,
	0.170,
	0.180
)

length <- c(
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010
)

data <- as.data.table(data.frame(var_a, var_b, slk_from, slk_to, length))
#data = data.frame(var_a,var_b,slk_from,slk_to,length)
data

var_a,var_b,slk_from,slk_to,length
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,0.00,0.01,0.01
2,2,0.01,0.02,0.01
3,1,0.02,0.03,0.01
4,3,0.03,0.04,0.01
2,5,0.04,0.05,0.01
3,6,0.05,0.06,0.01
1,5,0.06,0.07,0.01
5,6,0.07,0.08,0.01
3,4,0.08,0.09,0.01


In [3]:
var <- c("var_a", "var_b")

data_var <- data[, ..var]
data_var

var_a,var_b
<dbl>,<dbl>
1,3
2,2
3,1
4,3
2,5
3,6
1,5
5,6
3,4


In [4]:
min.length <- 0.020 # min(length)
max.length <- 0.100 # sum(length)

In [6]:
# find the range of indecies in the middle of the sequence
# such that if segmentation were performed at that index
# then the lengh of the segments on either side would be
# would be longer than the minimum length

data_length     <- data[["length"]] # array
cumlength_left  <- cumsum(data_length);
cumlength_right <- cumsum(rev(data_length))

k_left  <-                  c(1, which(cumlength_left  <= min.length) + 1)
k_left

k_right <- nrow(data_var) - c(0, which(cumlength_right <= min.length))
k_right

[1] 1 2 3

[1] 18 17 16

In [7]:
# note that negating the indecies has the effect of
# removing those indecies from the list
k <- c(1 : nrow(data_var))[-c(k_left, k_right)]
k

[1]  4  5  6  7  8  9 10 11 12 13 14 15

The following function computes the following:
$$
Q_s = 1 - \frac{N_a \sigma^{2}_{a} + N_b \sigma^{2}_{b}}{N^{2}_{\sigma}}
$$

In [17]:
cumq <- function(data){ # debug: use cumq to calculate q values along all segment points
    n                    <- length(data)
    cum_n                <- 1:(n - 1) # reduces length by 1
	stopifnot(length(cum_n) == length(data) - 1)
    data_left            <- data[cum_n]
    data_right           <- rev(data)[cum_n]
    cum_data_left        <- cumsum(data_left)
    cum_data_right       <- rev(cumsum(data_right))
    sumd                 <- sum(data)
    cum_datasquare_left  <- cumsum(data_left * data_left)
    cum_datasquare_right <- rev(cumsum(data_right * data_right))

    cqvalue <- (
        1 
        - (
            (cum_datasquare_left  - cum_data_left  * cum_data_left  / cum_n) +
            (cum_datasquare_right - cum_data_right * cum_data_right / rev(cum_n))
        ) / (sum(data * data) - sumd * sumd / n)
    )

    return(cqvalue)
}

In [18]:
qvalue <- matrix(NA, length(k), length(var))
for (i in 1 : length(var)) {
	qvalue[, i] <- cumq(data_var[[i]])[k - 1] # debug: use cumq function instead of q function
}
qvalue

0.2316742,0.1541401274
0.1709761,0.1642402184
0.2675948,0.0941695247
0.2992081,0.0286624204
0.5192455,0.0100090992
0.3896493,0.0003184713
0.4524887,0.0007077141
0.4380090,0.0385350318
0.2717283,0.2323599967
0.2737557,0.2579617834
0.3709711,0.2352768251


In [21]:

qvalue <- rowMeans(qvalue)
maxk <- which(qvalue == max(qvalue)) + max(k_left)

In [22]:
qvalue

[1] 0.1443267 0.1929072 0.1676082 0.1808822 0.1639353 0.2646273 0.1949839
 [8] 0.2265982 0.2382720 0.2520441 0.2658587 0.3031240 0.1603360 0.1318212

In [23]:
maxk

[1] 14

In [24]:
c(2:4)

[1] 2 3 4

In [25]:
cbind(c(1,2,3),c(4,5,6))

1,4
2,5
3,6


In [28]:
c(1:10)[-c(3,4)]

[1]  1  2  5  6  7  8  9 10

In [29]:
-c(3,4)

[1] -3 -4

In [4]:
rep(1:2,2)

[1] 1 2 1 2

In [19]:
c(1,2,3)[0]

numeric(0)